# Data Preparation
In this notebook, we use a subset of [Stack Exchange network](https://archive.org/details/stackexchange) question data which includes original questions tagged as 'JavaScript', their duplicate questions and their answers. Here, we provide the steps to prepare the data to use for training, tuning, and testing a model that will match a new question with an existing original question. The data files produced are stored in a `data` directory for ease of reference and also to keep them separate from the training script.

The data preparation steps are
- [import libraries and define parameters](#import),
- [ingest the data](#ingest),
- [cleanse the data](#cleanse),
- [prepare the train, tune, and test datasets](#prepare), and
- [save the datasets.](#save)

## Imports and parameters <a id='import'></a>

In [ ]:
import os
import pandas as pd
from text_utilities import read_csv_gz, clean_text, round_sample_strat, random_merge

Below, we define some parameters that will be used in the data cleaning as well as train and test set preparation.

In [ ]:
tune_size   = 0.10 # The proportion of duplicate questions in the tune set.
test_size   = 0.10 # The proportion of duplicate questions in the test set.
min_text    = 150  # The minimum length of clean text.
min_dupes   = 12   # The minimum number of duplicates per question.
match       = 40   # The number of duplicate matches.
output_path = os.path.join('.', 'data')  # The location of data files.

## Data ingestion <a id='ingest'></a>
Next, we download the questions, duplicate questions and answers and load the datasets into pandas dataframes using the helper functions.

Create URLs to original questions, duplicate questions, and answers.

In [ ]:
data_url = 'https://bostondata.blob.core.windows.net/stackoverflow/{}'
questions_url = data_url.format('orig-q.tsv.gz')
dupes_url = data_url.format('dup-q.tsv.gz')
answers_url = data_url.format('ans.tsv.gz')

Download the datasets.

In [ ]:
questions = read_csv_gz(questions_url, names=('Id', 'AnswerId', 'Text0', 'CreationDate'))
dupes = read_csv_gz(dupes_url, names=('Id', 'AnswerId', 'Text0', 'CreationDate'))
answers = read_csv_gz(answers_url, names=('Id', 'Text0'))

Let's now check the dataframes. Notice that both questions and duplicates have an "AnswerID" column used to match them with the index of the answers. Here are some of the original questions.

In [ ]:
questions.head()

The duplicate questions.

In [ ]:
dupes.head()

And the answers to the questions.

In [ ]:
answers.head()

Let's check the first original question's text.

In [ ]:
questions.Text0.iloc[0]

Let's now check the duplicates of that question.

In [ ]:
dupes[dupes.AnswerId == questions.AnswerId.iloc[0]]

Below is the answer to the original question.

In [ ]:
answers.Text0[questions.AnswerId.iloc[0]]

## Data cleaning <a id='cleanse'></a>
Next, we use a helper function to clean questions, duplicates, and answers of unwanted text such as code, html tags, and links. These clean texts in lowercase are added in new columns 'Text' in each dataframe.

In [ ]:
for df in (questions, dupes, answers):
    df['Text'] = df.Text0.apply(clean_text).str.lower()

Keep only rows with some clean text.

In [ ]:
questions = questions[questions.Text.str.len() > 0]
answers = answers[answers.Text.str.len() > 0]
dupes = dupes[dupes.Text.str.len() > 0]

Let's compare the first original question and cleaned version as an example. First, here's an original question.

In [ ]:
questions.Text0.iloc[0]

Here's the question after cleaning.

In [ ]:
questions.iloc[0,3]

It turns out that some duplicate questions were also in the original questions, and also some original questions and some duplicate questions had duplicates in their respective datasets. In the following, we remove them from the dataframes.

In [ ]:
dupes = dupes[~dupes.index.isin(questions.index)] # Remove dupes that are questions.
questions = questions[~questions.index.duplicated(keep='first')] # Then remove duplicates from the questions and duplicates.
dupes = dupes[~dupes.index.duplicated(keep='first')]

We also make sure we keep only questions with answers and duplicates, and answers and duplicates that have questions.

In [ ]:
questions = questions[questions.AnswerId.isin(answers.index) & questions.AnswerId.isin(dupes.AnswerId)]
answers = answers[answers.index.isin(questions.AnswerId)]
dupes = dupes[dupes.AnswerId.isin(questions.AnswerId)]

Verify the integrity of the remaining data.

In [ ]:
if not questions.AnswerId.isin(answers.index).all():
    raise Exception('Not all original questions have answers')
if not answers.index.isin(questions.AnswerId).all():
    raise Exception('Not all answers have original questions.')
if not questions.AnswerId.isin(dupes.AnswerId).all():
    raise Exception('Not all original questions have duplicates.')
if not dupes.AnswerId.isin(questions.AnswerId).all():
    raise Exception('Not all duplicates have original questions.')

Below are counts of unique clean texts in each dataframe, and statistics on the lengths of those texts. There are also statistics on the number of duplicates available for each question. Notice that some questions have only a few duplicates while others have a large number. 

In [ ]:
print('Text statistics:')
print(pd.DataFrame([questions.Text.str.len().describe()
                    .rename('questions'),
                    answers.Text.str.len().describe()
                    .rename('answers'),
                    dupes.Text.str.len().describe()
                    .rename('dupes')]))
print('\nDuplication statistics:')
print(pd.DataFrame([dupes.AnswerId.value_counts().describe()
                    .rename('duplications')]))
print('\nLargest class: {:.2%}'
      .format(dupes.AnswerId.value_counts().max()
              / dupes.shape[0]))

Now, we reset all dataframe indexes to use them as columns in the remaining steps.

In [ ]:
questions.reset_index(inplace=True)
answers.reset_index(inplace=True)
dupes.reset_index(inplace=True)

We filter the questions and duplicates to have at least min_text number of characters.

In [ ]:
questions = questions[questions.Text.str.len() >= min_text]
dupes = dupes[dupes.Text.str.len() >= min_text]

Again, keep only questions with dupes and dupes of questions.

In [ ]:
questions = questions[questions.AnswerId.isin(dupes.AnswerId)]
dupes = dupes[dupes.AnswerId.isin(questions.AnswerId)]

Here, we keep questions that have at least min_dupes duplicates, and then keep only those duplicates.

In [ ]:
answerid_count = dupes.groupby('AnswerId').AnswerId.count()      # Count the number of duplicates by AnswerId.
answerid_min = answerid_count.index[answerid_count >= min_dupes] # Find the AnswerIds with at least min_dupes duplicates.
questions = questions[questions.AnswerId.isin(answerid_min)]     # Keep only questions with those AnswerIds.
dupes = dupes[dupes.AnswerId.isin(answerid_min)]                 # Keep only dupes with those AnswerIds.

Once again, verify data integrity.

In [ ]:
if not questions.AnswerId.isin(dupes.AnswerId).all():
    raise Exception('Not all original questions have duplicates.')
if not dupes.AnswerId.isin(questions.AnswerId).all():
    raise Exception('Not all duplicates have original questions.')

Here are some statistics on the resulting dataset.

In [ ]:
print('Restrictions: min_text={}, min_dupes={}'
      .format(min_text, min_dupes))
print('Restricted text statistics:')
print(pd.DataFrame([questions.Text.str.len().describe()
                    .rename('questions'),
                    dupes.Text.str.len().describe()
                    .rename('dupes')]))
print('\nRestricted duplication statistics:')
print(pd.DataFrame([dupes.AnswerId.value_counts().describe()
                    .rename('duplications')]))
print('\nRestricted largest class: {:.2%}'
      .format(dupes.AnswerId.value_counts().max()
              / dupes.shape[0]))

## Prepare train, tune, and test sets <a id='prepare'></a>

In this part, we prepare train, tune, and test sets. For training a binary classification model, we will need to construct match and non-match pairs from duplicates and their questions. Finding matching pairs can be accomplished by joining each duplicate with its question. However, non-match examples need to be constructed randomly.

As a first step, to make sure we train and test the performance of the model on each question, we will need to have examples of match and non-match pairs for each question both in train and test sets. In order to achieve that, we split the duplicates in a stratified manner into train, tune, and test sets making sure at least 1 or more duplicates per question is in both the tune and tests set depending on the tune_size and test_size parameters and number of duplicates per each question.

In [ ]:
dupes_test = round_sample_strat(dupes, dupes.AnswerId, frac=test_size)
dupes_train = dupes[~dupes.Id.isin(dupes_test.Id)]
if not (dupes_test.AnswerId.unique().shape[0] == dupes.AnswerId.unique().shape[0]):
    raise Exception('The number of unique questions in dupes_test is not equal to those in dupes.')

dupes_tune = round_sample_strat(dupes_train, dupes_train.AnswerId, frac=tune_size)
dupes_train = dupes_train[~dupes_train.Id.isin(dupes_tune.Id)]
if not (dupes_tune.AnswerId.unique().shape[0] == dupes_train.AnswerId.unique().shape[0]):
    raise Exception('The number of unique questions in dupes_tune is not equal to those in dupes_train.')

Report on the number of duplicate questions in each dataset.

In [ ]:
print("dupes_train has {:,} questions".format(dupes_train.shape[0]))
print("dupes_tune has {:,} questions".format(dupes_tune.shape[0]))
print("dupes_test has {:,} questions".format(dupes_test.shape[0]))

The names we will use for the relevant columns for text pairs data.

In [ ]:
balanced_pairs_columns = ['Id_x', 'AnswerId_x', 'Text_x', 'Id_y', 'Text_y', 'AnswerId_y', 'Label', 'n']

Next, we use AnswerId to pair each training duplicate in train set with its matching question and N-1 random questions using the helper function.

In [ ]:
%time balanced_pairs_train = random_merge(dupes_train, questions, N=match)

Create a label for each row such that matching pairs are labeled as 1 and non-matching pairs are labeled as 0.

In [ ]:
balanced_pairs_train['Label'] = (balanced_pairs_train.AnswerId_x == balanced_pairs_train.AnswerId_y).astype(int)

Keep only the relevant columns.

In [ ]:
balanced_pairs_train = balanced_pairs_train[balanced_pairs_columns]

Sort the data by the dupe ID and the Label.

In [ ]:
balanced_pairs_train.sort_values(by=['Id_x', 'Label'], ascending=[True, False], inplace=True)

Here are the first few rows of the training data.

In [ ]:
balanced_pairs_train.head()

For the tune and test sets, we match each duplicate with _all_ the original questions.

In [ ]:
%time balanced_pairs_tune = random_merge(dupes_tune, questions, N=questions.shape[0])
%time balanced_pairs_test = random_merge(dupes_test, questions, N=questions.shape[0])

Label the rows in same way as was done for the training set.

In [ ]:
balanced_pairs_tune['Label'] = (balanced_pairs_tune.AnswerId_x == balanced_pairs_tune.AnswerId_y).astype(int)
balanced_pairs_test['Label'] = (balanced_pairs_test.AnswerId_x == balanced_pairs_test.AnswerId_y).astype(int)

Keep only the relevant columns.

In [ ]:
balanced_pairs_tune = balanced_pairs_tune[balanced_pairs_columns]
balanced_pairs_test = balanced_pairs_test[balanced_pairs_columns]

Sort the data by dupe ID and Label.

In [ ]:
balanced_pairs_tune.sort_values(by=['Id_x', 'Label'], ascending=[True, False], inplace=True)
balanced_pairs_test.sort_values(by=['Id_x', 'Label'], ascending=[True, False], inplace=True)

Here are the first few rows of the test data. The tune data are similar.

In [ ]:
balanced_pairs_test.head()

We report statistics the train and test sets.

In [ ]:
print('balanced_pairs_train: {:,} rows with {:.2%} matches'
      .format(balanced_pairs_train.shape[0], 
              balanced_pairs_train.Label.mean()))
print('balanced_pairs_tune: {:,} rows with {:.2%} matches'
      .format(balanced_pairs_tune.shape[0], 
              balanced_pairs_tune.Label.mean()))
print('balanced_pairs_test: {:,} rows with {:.2%} matches'
      .format(balanced_pairs_test.shape[0], 
              balanced_pairs_test.Label.mean()))

## Save the datasets <a id='save'></a>
Finally, we save as text files the questions, the train and test duplicates, and the train and test dataframes of duplicate-question pairs.

In [ ]:
os.makedirs(output_path, exist_ok=True)

questions_path = os.path.join(output_path, 'questions.tsv')
print('Writing {:,} rows to {}'.format(questions.shape[0], questions_path))
questions.to_csv(questions_path, sep='\t',header=True, index=False)

answers_path = os.path.join(output_path, 'answers.tsv')
print('Writing {:,} rows to {}'.format(answers.shape[0], answers_path))
answers.to_csv(answers_path, sep='\t',header=True, index=False)

dupes_train_path = os.path.join(output_path, 'dupes_train.tsv')
print('Writing {:,} rows to {}'.format(dupes_train.shape[0], dupes_train_path))
dupes_train.to_csv(dupes_train_path, sep='\t',header=True, index=False)

dupes_tune_path = os.path.join(output_path, 'dupes_tune.tsv')
print('Writing {:,} rows to {}'.format(dupes_tune.shape[0], dupes_tune_path))
dupes_tune.to_csv(dupes_tune_path, sep='\t',header=True, index=False)

dupes_test_path = os.path.join(output_path, 'dupes_test.tsv')
print('Writing {:,} rows to {}'.format(dupes_test.shape[0], dupes_test_path))
dupes_test.to_csv(dupes_test_path, sep='\t',header=True, index=False)

balanced_pairs_train_path = os.path.join(output_path, 'balanced_pairs_train.tsv')
print('Writing {:,} rows to {}'.format(balanced_pairs_train.shape[0], balanced_pairs_train_path))
balanced_pairs_train.to_csv(balanced_pairs_train_path, sep='\t',header=True, index=False)

balanced_pairs_tune_path = os.path.join(output_path, 'balanced_pairs_tune.tsv')
print('Writing {:,} rows to {}'.format(balanced_pairs_tune.shape[0], balanced_pairs_tune_path))
balanced_pairs_tune.to_csv(balanced_pairs_tune_path, sep='\t', header=True, index=False)

balanced_pairs_test_path = os.path.join(output_path, 'balanced_pairs_test.tsv')
print('Writing {:,} rows to {}'.format(balanced_pairs_test.shape[0], balanced_pairs_test_path))
balanced_pairs_test.to_csv(balanced_pairs_test_path, sep='\t', header=True, index=False)

We can now move on to [defining the model training script](01_Training_Script.ipynb).